In [3]:
%pip install langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn
ERROR: Could not find a version that satisfies the requirement langchain (from versions: none)
ERROR: No matching distribution found for langchain
Note: you may need to restart the kernel to use updated packages.


## LLM Chain

In [4]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [5]:
llm.invoke("how can langsmith help with testing?")

'\nLangsmith is a tool that can assist with testing by providing various features and functionalities. Here are some ways Langsmith can help with testing:\n\n1. Automated Testing: Langsmith provides an API for automating tests, which can be used to create and run test cases automatically. This can save time and reduce the likelihood of human error.\n2. Code Analysis: Langsmith can analyze your codebase and identify potential issues, such as duplicate code, unused variables, or suspicious expressions. This can help you identify areas that need testing.\n3. Test Case Generation: Langsmith can generate test cases based on your codebase and requirements. This can help you create comprehensive test suites and reduce the time and effort required to write tests.\n4. Test Data Generation: Langsmith can also generate test data, such as input values or test cases, based on your requirements. This can help you create more comprehensive test suites and reduce the time and effort required to create

We can also guide it's response with a prompt template. Prompt templates are used to convert raw user input to a better input to the LLM.

In [6]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

We can now combine these into a simple LLM chain:

In [7]:
chain = prompt | llm

We can now invoke it and ask the same question. It still won't know the answer, but it should respond in a more proper tone for a technical writer!

In [8]:
chain.invoke({"input": "how can langsmith help with testing?"})

"\nAh, an excellent question! As a world-class technical documentation writer, I must say that Langsmith has a plethora of features that can aid in the testing process. Here are some ways in which Langsmith can help:\n\n1. Automated Content Analysis: With Langsmith's AI-powered content analysis tools, you can easily identify areas of your documentation that require improvement. For instance, you can analyze your documentation for consistency, accuracy, and completeness, and receive actionable insights on how to improve it.\n2. Quality Assurance: Langsmith's QA features enable you to test your documentation thoroughly before publishing. You can identify and fix errors, inconsistencies, and other issues that might affect the user experience. By doing so, you can ensure that your documentation is of the highest quality and provides accurate information to your users.\n3. Collaboration: Langsmith allows multiple authors to work together on a single documentation set. This enables you to st